In [2]:
# mount the google drive path
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/Colab Notebooks/face-classification

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/face-classification


In [5]:
!git add .

In [6]:
git config --global user.email "zin"
  git config --global user.name "Your Name"
!git commit -am "added notebook"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@32237e8f6172.(none)')


In [ ]:
!git push 